In [ ]:
!pwd

In [ ]:
%cd ~/Documents/Glasses_Detection_with_YOLOv5/yolov5

In [ ]:
import os
import torch
import torch.nn.utils.prune as prune
from pathlib import Path
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import gc
from utils.datasets import create_dataloader
from utils.general import (LOGGER, box_iou, check_dataset, check_img_size, check_requirements, check_yaml,
                           coco80_to_coco91_class, colorstr, increment_path, non_max_suppression, print_args,
                           scale_coords, xywh2xyxy, xyxy2xywh)
from utils.metrics import ConfusionMatrix, ap_per_class
from utils.plots import output_to_target, plot_images, plot_val_study
from utils.torch_utils import select_device, time_sync
import torch_pruning as tp

In [ ]:
device = "cuda:0"

In [ ]:
def apply_prune(model, pruning_ratio=0.3, real_pruning=False, mode='Structured'):
    print('Pruning model with ratio = {}... '.format(pruning_ratio), end='')
    if mode == 'Structured':
        pruning_ratio = 1 - np.sqrt(1 - pruning_ratio) # for removing amount of parameters / structured pruning removes channel
    if not real_pruning:
        for name, m in model.named_modules():
            if isinstance(m, torch.nn.Conv2d):
                if mode == 'Structured':
                    prune.ln_structured(m, name='weight', amount=pruning_ratio, n=1, dim=1)  # prune
                elif mode == 'Unstructured':
                    prune.l1_unstructured(m, name='weight', amount=pruning_ratio)  # prune
                prune.remove(m, 'weight')  # make permanent
                
        for p in model.parameters():
            p.requires_grad_(True)
    else:
        #print(model.model)
        for p in model.parameters():
            p.requires_grad_(True)
        example_inputs = torch.randn(1, 3, 640, 640).to(device)
    
        ignored_layers = []
        from models.yolo import Detect
        for m in model.model.modules():
            if isinstance(m, Detect):
                ignored_layers.append(m)
        #print(ignored_layers)

        iterative_steps = 1 # progressive pruning
        pruner = tp.pruner.GroupNormPruner(
            model.model,
            example_inputs,
            importance=tp.importance.GroupNormImportance(p=1, normalizer=None, group_reduction='sum'),
            global_pruning=False,
            isomorphic=False,
            iterative_steps=iterative_steps,
            pruning_ratio=pruning_ratio, # remove 50% channels, ResNet18 = {64, 128, 256, 512} => ResNet18_Half = {32, 64, 128, 256}
            ignored_layers=ignored_layers,
        )
        pruner.step()
    print('Pruning end. Calculating model information...')
    return model

def get_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    model_size = os.path.getsize("tmp.pt")/1e6
    print("model_size: %.2f MB" %(model_size))
    os.remove('tmp.pt')
    return model_size

def flops_and_params(model):
    example_inputs = torch.randn(1, 3, 640, 640).to(device)
    macs, nparams = tp.utils.count_ops_and_params(model, example_inputs)
    flops = 2 * macs
    return flops, nparams

In [ ]:
def process_batch(detections, labels, iouv):
    """
    Return correct predictions matrix. Both sets of boxes are in (x1, y1, x2, y2) format.
    Arguments:
        detections (Array[N, 6]), x1, y1, x2, y2, conf, class
        labels (Array[M, 5]), class, x1, y1, x2, y2
    Returns:
        correct (Array[N, 10]), for 10 IoU levels
    """
    correct = torch.zeros(detections.shape[0], iouv.shape[0], dtype=torch.bool, device=iouv.device)
    iou = box_iou(labels[:, 1:], detections[:, :4])
    x = torch.where((iou >= iouv[0]) & (labels[:, 0:1] == detections[:, 5]))  # IoU above threshold and classes match
    if x[0].shape[0]:
        matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()  # [label, detection, iou]
        if x[0].shape[0] > 1:
            matches = matches[matches[:, 2].argsort()[::-1]]
            matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
            # matches = matches[matches[:, 2].argsort()[::-1]]
            matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
        matches = torch.Tensor(matches).to(iouv.device)
        correct[matches[:, 1].long()] = matches[:, 2:3] >= iouv
    return correct

def evaluate(model, data_yaml, device='cuda:0', img_size=640, batch_size=32, conf_thres=0.001, iou_thres=0.6, half=True):
    """
    YOLOv5 모델 테스트 함수 (2022년 초 버전 기반).
    
    Args:
        model (torch.nn.Module): PyTorch YOLOv5 모델 객체
        data_yaml (str or Path): 데이터셋 yaml 파일 경로
        device (str): 'cuda' 또는 'cpu'
        img_size (int): 입력 이미지 크기 (정사각형 기준)
        batch_size (int): 배치 크기
        conf_thres (float): Confidence threshold
        iou_thres (float): IoU threshold for NMS
    
    Returns:
        dict: 테스트 결과 메트릭 (Precision, Recall, mAP@0.5, mAP@0.5:0.95)
    """
    # 디바이스 설정
    device = torch.device(device)

    # 데이터셋 확인
    data = check_dataset(data_yaml)
    
    model.eval()
    is_coco = isinstance(data.get('val'), str) and data['val'].endswith('coco/val2017.txt')  # COCO dataset
    nc = int(data['nc'])  # number of classes
    iouv = torch.linspace(0.5, 0.95, 10).to(device)  # iou vector for mAP@0.5:0.95
    niou = iouv.numel()
    
    pad = 0.5
    task = 'val'  # path to train/val/test images
    dataloader = create_dataloader(data[task], img_size, batch_size, model.stride, False, pad=pad, rect=True,
                                    workers=8, prefix=colorstr(f'{task}: '))[0]
    
    seen = 0
    names = {k: v for k, v in enumerate(model.names if hasattr(model, 'names') else model.module.names)}
    class_map = list(range(1000))
    s = ('%20s' + '%11s' * 6) % ('Class', 'Images', 'Labels', 'P', 'R', 'mAP@.5', 'mAP@.5:.95')
    dt, p, r, f1, mp, mr, map50, map = [0.0, 0.0, 0.0], 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    loss = torch.zeros(3, device=device)
    jdict, stats, ap, ap_class = [], [], [], []
    pbar = tqdm(dataloader, desc=s, bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')  # progress bar
    
    for batch_i, (im, targets, paths, shapes) in enumerate(pbar):
        t1 = time_sync()
        im = im.to(device, non_blocking=True)
        targets = targets.to(device)
        im = im.half() if half else im.float()  # uint8 to fp16/32
        im /= 255  # 0 - 255 to 0.0 - 1.0
        nb, _, height, width = im.shape  # batch size, channels, height, width
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        out = model(im, augment=False)  # inference, loss outputs
        dt[1] += time_sync() - t2

        # NMS
        targets[:, 2:] *= torch.Tensor([width, height, width, height]).to(device)  # to pixels
        lb = []  # for autolabelling
        t3 = time_sync()
        out = non_max_suppression(out, conf_thres, iou_thres, labels=lb, multi_label=True, agnostic=False)
        dt[2] += time_sync() - t3

        # Metrics
        for si, pred in enumerate(out):
            labels = targets[targets[:, 0] == si, 1:]
            nl = len(labels)
            tcls = labels[:, 0].tolist() if nl else []  # target class
            path, shape = Path(paths[si]), shapes[si][0]
            seen += 1

            if len(pred) == 0:
                if nl:
                    stats.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
                continue

            # Predictions
            predn = pred.clone()
            scale_coords(im[si].shape[1:], predn[:, :4], shape, shapes[si][1])  # native-space pred

            # Evaluate
            if nl:
                tbox = xywh2xyxy(labels[:, 1:5])  # target boxes
                scale_coords(im[si].shape[1:], tbox, shape, shapes[si][1])  # native-space labels
                labelsn = torch.cat((labels[:, 0:1], tbox), 1)  # native-space labels
                correct = process_batch(predn, labelsn, iouv)
                
            else:
                correct = torch.zeros(pred.shape[0], niou, dtype=torch.bool)
            stats.append((correct.cpu(), pred[:, 4].cpu(), pred[:, 5].cpu(), tcls))  # (correct, conf, pcls, tcls)

    # Compute metrics
    stats = [np.concatenate(x, 0) for x in zip(*stats)]  # to numpy
    if len(stats) and stats[0].any():
        tp, fp, p, r, f1, ap, ap_class = ap_per_class(*stats, names=names)
        ap50, ap = ap[:, 0], ap.mean(1)  # AP@0.5, AP@0.5:0.95
        mp, mr, map50, map = p.mean(), r.mean(), ap50.mean(), ap.mean()
        nt = np.bincount(stats[3].astype(np.int64), minlength=nc)  # number of targets per class
    else:
        nt = torch.zeros(1)

    # Print results
    pf = '%20s' + '%11i' * 2 + '%11.3g' * 4  # print format
    print(pf % ('all', seen, nt.sum(), mp, mr, map50, map))

    # Print speeds
    t = tuple(x / seen * 1E3 for x in dt)  # speeds per image
    shape = (batch_size, 3, img_size, img_size)
    print(f'Speed: %.1fms pre-process, %.1fms inference, %.1fms NMS per image at shape {shape}' % t)

    # 결과 반환
    results = {
        'Precision': mp,
        'Recall': mr,
        'mAP@0.5': map50,
        'mAP@0.5:0.95': map
    }
    return (results, t)

# Example Usage (adjust paths as needed):
# test_yolov5_v1(model, data_yaml='data/coco128.yaml', device='cuda')

In [ ]:
def evaluate_pruning(weight='n', real_pruning=False, mode='Unstructured', half=False):
    pruning_ratios = np.linspace(0.0, 0.9, 10)  # 0%, 10%, ..., 50%
    results = []
    inform = []

    model = torch.hub.load('.', 'custom', '../finetuned_weights/yolov5{}_finetuned.pt'.format(weight), source='local', force_reload=True, device=device)

    for i, ratio in enumerate(pruning_ratios):
        print('({}) pruning ratio: {}'.format(i+1, ratio))
        pruned_model = apply_prune(model, pruning_ratio=ratio, real_pruning=real_pruning, mode=mode)
        flops, nparams = flops_and_params(pruned_model)
        print('FLOPs: %.3f GFLOPs, number of parameters: %d' % (flops / 1e9, nparams))
        model_size = get_model_size(pruned_model)
        metrics, times = evaluate(model=model, data_yaml='./data/glasses.yaml', device=device, half=half)
        results.append([ratio, metrics['Precision'], metrics['Recall'], metrics['mAP@0.5'], metrics['mAP@0.5:0.95'], times])
        inform.append([model_size, flops, nparams])
        print('pruning ratio {} complete'.format(ratio))
    
    del model
    gc.collect()
    
    return results, inform

In [ ]:
%matplotlib inline
#results, informs = evaluate_pruning(weight=weight, real_pruning=False, mode='Unstructured')

def plot_results(weight, results, informs, real_pruning, mode):
    folder = '../pruning_results_2'
    # 데이터 추출
    ratios = [r[0] for r in results]
    precision_score = [r[1] for r in results]
    recall_score = [r[2] for r in results]
    map50_scores = [r[3] for r in results]
    map50_95_scores = [r[4] for r in results]
    inference_times = [r[5][1] for r in results]

    model_sizes = [i[0] for i in informs]
    flops = [i[1] for i in informs]
    nparams = [i[2] for i in informs]

    # 그래프 그리기
    fig, ax = plt.subplots(1, 3, figsize=(24,6))
    ax[0].plot(ratios, map50_scores, marker='o', label='mAP@0.5')
    ax[0].plot(ratios, map50_95_scores, marker='o', label='mAP@0.5:0.95')
    ax[0].set_title("mAP")
    ax[0].set_xlabel("Pruning Ratio")
    ax[0].set_ylabel("Performance")
    ax[0].legend()
    ax[0].grid(True)

    ax[1].plot(ratios, precision_score, color='r', marker='o', label='precision')
    ax[1].plot(ratios, recall_score, color='g', marker='o', label='recall')
    ax[1].set_title("Precision and Recall")
    ax[1].set_xlabel("Pruning Ratio")
    ax[1].set_ylabel("Performance")
    ax[1].legend()
    ax[1].grid(True)

    ax[2].plot(ratios, inference_times, color='m', marker='o', label='precision')
    ax[2].set_title("Inference Time")
    ax[2].set_xlabel("Pruning Ratio")
    ax[2].set_ylabel("time (ms)")
    ax[2].legend()
    ax[2].grid(True)

    fig.suptitle('YOLOv5{} {} {} Pruning Ratio vs Performance'.format(weight, 'Real' if real_pruning else 'Fake', mode))
    fig.tight_layout()
    plt.show()
    os.makedirs(folder, exist_ok=True)
    plt.savefig("{}/YOLOv5{}_{}_{}_Performance.png".format(folder, weight, 'Real' if real_pruning else 'Fake', mode))

    fig, ax = plt.subplots(1, 3, figsize=(24,6))
    ax[0].plot(ratios, model_sizes, 'bo-', label='model_size')
    ax[0].set_title("Model Size (MB)")
    ax[0].set_xlabel("Pruning Ratio")
    ax[0].set_ylabel("MB")
    ax[0].set_ylim(0, 1.4 * model_sizes[0])
    ax[0].legend()
    ax[0].grid(True)

    ax[1].plot(ratios, flops, 'ro-', label='GFLOPs')
    ax[1].set_title("FLOPs (G)")
    ax[1].set_xlabel("Pruning Ratio")
    ax[1].set_ylabel("GFLOPs")
    ax[1].legend()
    ax[1].grid(True)

    ax[2].plot(ratios, nparams, 'go-', label='number of parameters(M)')
    ax[2].set_title("Number of parameters (M)")
    ax[2].set_xlabel("Pruning Ratio")
    ax[2].set_ylabel("# parameters (M)")
    ax[2].legend()
    ax[2].grid(True)

    fig.suptitle('YOLOv5{} {} {} Pruning Ratio vs Information'.format(weight, 'Real' if real_pruning else 'Fake', mode))
    fig.tight_layout()
    plt.show()
    os.makedirs(folder, exist_ok=True)
    plt.savefig("{}/YOLOv5{}_{}_{}_Information.png".format(folder, weight, 'Real' if real_pruning else 'Fake', mode))

In [ ]:
weight = 'n'
exp_list = [(False, 'Unstructured'), (False, 'Structured'), (True, 'Structured')]

for real_pruning, mode in exp_list:
    results, informs = evaluate_pruning(weight, real_pruning, mode)
    plot_results(weight, results, informs, real_pruning, mode)